# rinna GPT-2 モデルの Fine Tuning
HuggingFace の transformers ライブラリを用いて [rinna gpt-2](https://huggingface.co/rinna/japanese-gpt2-medium) モデルの Fine Tuning を行います。

## 事前準備
必要なライブラリをインポートします。

In [1]:
from azureml.core import Experiment, Workspace, Environment
from azureml.core.compute import ComputeTarget
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration

import os
os.makedirs('src', exist_ok=True)

Azure ML Workspace へ接続します。

In [2]:
ws = Workspace.from_config()

実験 Experiment の名称

In [3]:
model_experiment = Experiment(ws, name="rinna-gpt2-exp")

PyTorch の設定

In [4]:
distr_config = PyTorchConfiguration()

環境 Environment の設定

In [5]:
hf_ort_env = Environment.from_dockerfile(name='rinna-docker-env', dockerfile='Dockerfile')
#hf_ort_env.build(ws)

スクリプトの引数の定義

In [6]:
script_params = ['--num_train_epochs', 1,'--output_dir', './outputs', '--model_name_or_path', 'rinna/japanese-gpt2-medium']

## モデル学習
`ScriptRunConfig` を用いて Azure Machine Learning Compute Cluster 上で学習ができるように設定します。

In [ ]:
model_run_config = ScriptRunConfig(
    source_directory='./src',
    script='./train.py',
    arguments=script_params,
    compute_target=ComputeTarget(workspace=ws, name="gpuinstance"),
    environment=hf_ort_env,
    distributed_job_config=distr_config)

モデル学習の開始

In [ ]:
run = model_experiment.submit(model_run_config)
run

Experiment,Id,Type,Status,Details Page,Docs Page
rinna-gpt2-exp,rinna-gpt2-exp_1628239017_d4034abd,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [105]:
#run.wait_for_completion(show_output=False)